# Summary of using qcodes with the nplab_drivers folder

### Last updated: 7/31/2019

## Introduction

This serves as an introduction to using QCodes (relatively efficiently), given that you've figured out the installation of QCodes and my nplab_drivers folder. This is aimed toward someone who has never used QCodes. However, it avoids covering design of instrument drivers and many of the details behind the code, in the interest of conciseness. All blocks here were designed to be run on any machine, without requiring a connection to special instruments.

I made this folder (to be placed in the qcodes/instrument_drivers subfolder) to simplify the use of QCodes. I wanted a customizable but relatively simple set of commands to modularize the use of QCodes for people who don't want to memorize more than a few commands. Note, this is based on the first QCodes loop/database structure, called "legacy QCodes" in the current QCodes documentation. There are 3 basic types of measurements that most measurements fit into. I've made a function for each:

###   1. single_param_sweep
        Sweep a single parameter through an array of setpoints, while measuring other parameters
###   2. twod_param_sweep
        Sweep two parameters (x and y axes) through arrays of setpoints, while measuring other parameters (z axis)
###   3. data_log
        Log parameters periodically. Used for measuring, for instance, a temperature or magnetic field sweep to a 
        setpoint. Set the parameter right before running the log and measure the results over time.

Each of these includes the direct implementation of live plotting, which parameters to plot, the appropriate delay times, and the choice of a name to be attached to the data file. They each automatically save the measured data to a file. Each one of these commands can be found in the common_commands.py file.

The main idea of QCodes comes through allowing us to write and read values through "parameters". Note a "parameter" in QCodes takes on a few technical connotations. Most often parameters will be attributes of an instrument and will be loaded along with the instrument. You can set a parameter to a value using parentheses, with a single value inside. For example (using an instrument called "k2000" and a parameter called "current", setting current to 10 mA): `k2000.current(0.01)`

You can get/measure a parameter using open parentheses. For instance: `k2000.current()`


**Footnotes:** These measurements were specifically designed for parameters that return a single numerical result per measurement. QCodes allows you to take a measurement that returns something more complex, such as an array, a categorical or string result, or a combination of types of results. These can still be measured using the nplab_drivers approach, but plotting likely will not work.

There is actually another type of measurement you may want to do: a one-shot measurement that saves to file. That one is covered already by qc.Measure(parameters, separated, by, commas).run()

## Import as follows

In [1]:
import numpy as np  # numpy for handling arrays
import time  # time provides a useful sleep function
import qcodes as qc

# Import the functions from the NP lab drivers directory
import qcodes.instrument_drivers.nplab_drivers as npd

# To import instruments from the qcodes registry, import the class that has the name of your instrument type
from qcodes.instrument_drivers.tektronix.Keithley_2000 import Keithley_2000

# To import my custom instruments from the NP lab directory
from qcodes.instrument_drivers.nplab_drivers.Keithley_6221 import Keithley_6221

Then instantiate the instruments using "name = Instrument_class(name string, address string)"
for instance:

`k2000 = Keithley_2000('k2000', 'GPIB::0::INSTR')`


You can also set up parameters without an instrument, if you need. For instance:

`pp = Parameter('pp', get_cmd=get, set_cmd=set)`

where "get" is a reference to a function that returns a value when used with open parentheses.
"set" is a a reference to a function that takes one value in parentheses and returns nothing. If you don't use a get_cmd argument, using the get function will return an error, and similarly for the set_cmd argument.

## Set up some dummy parameters to set and measure

(Don't worry too much about the details of setting these up. Most parameters will be defined for you already through the instrument drivers, so you can skip the confusing details here until you want more info.)

In [2]:
pmanual = qc.Parameter('pmanual', get_cmd=None, set_cmd=None)  # A simple dummy parameter can be set up by using None in the set_cmd and get_cmd arguments.
# This allows you to set the parameter to whatever you want and retrieve the value you set it to.

In [3]:
pmanual(5.5)  # set the parameter

In [4]:
pmanual()  # get the parameter (returns what you set it to)

5.5

Now I'll set up a parameter that depends on pmanual (to simulate an actual measurement)

In [5]:
def squareval():
    return pmanual()**2

p1 = qc.Parameter('p1', get_cmd=squareval, unit='V')  # Note you can build a parameter out of other parameters.
# This could be useful if you want to, for instance, define a resistance parameter as voltage()/current()

In [6]:
p1()

30.25

And I'll set up a dummy instrument to simulate measuring parameters from an instrument

In [7]:
class myinstr(qc.Instrument):
    """An instrument with 2 parameters:
            c1: counts the number of times the parameter has been called up to this point
            c2: returns the square root of the number of times the parameter has been called
            p2: returns the sum of the two counts and the value of pmanual. doesn't advance the counts"""
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)  # initializes the sequence included in the qc.Instrument class
        self._count1 = 0  # a way to pass values within an instrument (not a parameter)
        self._count2 = 0  # I just put an underscore out front to symbolize that it is sort of hidden
        
        self.add_parameter('c1', get_cmd=self.countup)
        self.add_parameter('c2', get_cmd=self.countsqrt, label='Square root of count')  # label will show up in plots
        self.add_parameter('p2', get_cmd=self.addcounts_andp)
        
    def countup(self):
        self._count1 += 1
        return self._count1
    
    def countsqrt(self):
        self._count2 += 1
        return np.sqrt(self._count2)
    
    def addcounts_andp(self):
        return pmanual() + self._count1 + self._count2
    
    def resetcounts(self):  # this is just an example of an instrument function (not a parameter, so can't be measured)
        self._count1 = 0
        self._count2 = 0

In [8]:
# instantiate the instrument
instr1 = myinstr('instr1')

In [9]:
instr1.c1()

1

In [10]:
instr1.c1()

2

In [11]:
instr1.c2()

1.0

In [12]:
instr1.c2()

1.4142135623730951

In [13]:
instr1.p2()

9.5

### A few other supplementary notes about parameters:

You can cause a parameter (when one uses the set command) to advance in steps, with delay times between each step. That way you don't cause parameters to jump rapidly, causing device damage or other problems.

In [14]:
pmanual.step = 1
pmanual.inter_delay = 0.3

Using these settings, if I set pmanual from 5.5 to 10, the value will approach 10 in jumps of size 1, with a minimum time between jumps of half a second.

In [15]:
pmanual(10)  # note the lag time. The set command will not finish until the ramp is done.
# Measurements will not record the intermediate values.

If re-measuring the value takes time or messes with the setup, you can use the last-retrieved result

In [16]:
pmanual.get_latest()

10

## 1. Demonstration of single_param_sweep

Structure:

`returned_data, plot_list = npd.single_param_sweep(SetParam, SetArray, delay(s), *Measured_Params)`

Where `*Measured_Params` refers to comma-separated parameters (as many as you want). `delay` is the delay time after setting the set parameter.

This will automatically plot all the measured parameters.

In [17]:
setarr = np.linspace(0, 20, 6)
data1, plot1 = npd.single_param_sweep(pmanual, setarr, 1.2, p1, instr1.c1, instr1.c2)

Started at 2019-07-31 02:12:23
DataSet:
   location = 'data/2019-07-31/#001__02-12-18'
   <Type>   | <array_id>  | <array.name> | <array.shape>
   Setpoint | pmanual_set | pmanual      | (6,)
   Measured | p1          | p1           | (6,)
   Measured | instr1_c1   | c1           | (6,)
   Measured | instr1_c2   | c2           | (6,)
Finished at 2019-07-31 02:12:38


This should automatically generate 3 plots live and store the data in a folder ./data/{date}/{dataset_name}, if everything is working right. There are two files: a snapshot of parameters before the sweep was performed, and the dataset text file (.dat extension). You can stop this, and any other sweep, with the stop button on Jupyter Notebook, and it will save the previously measured data.

(If you can't find the location of this data folder, try `qc.DataSet.default_io.base_location` to find where it's hiding. You can also change this location by using `qc.DataSet.default_io.base_location = "path/to/desired/location"`.)

This demonstrates the other main idea of QCodes (secondary to settable and gettable parameters) is a method to store data in a "dataset" that is organized by date/time and cannot be changed easily (since post-processing of data is assumed to be handled separately). This dataset was returned in the previous function as `data1`. We can retrieve the arrays of set and measured parameters by using the names listed under the <array_id> column.

In [18]:
data1.p1

DataArray[6]: p1
array([  0.,  16.,  64., 144., 256., 400.])

In [19]:
data1.instr1_c1

DataArray[6]: instr1_c1
array([3., 4., 5., 6., 7., 8.])

In [20]:
data1.pmanual_set

DataArray[6]: pmanual_set
array([ 0.,  4.,  8., 12., 16., 20.])

In [21]:
data1.instr1_c1*5

TypeError: unsupported operand type(s) for *: 'DataArray' and 'int'

In [22]:
data1.instr1_c1[:]*5

array([15., 20., 25., 30., 35., 40.])

In [23]:
np.array(data1.instr1_c1)*5

array([15., 20., 25., 30., 35., 40.])

Make a note of a few things with regard to retrieving results from datasets. When using an instrument, you must use `data.instr_param` **instead of** `data.instr.param`, since this is how it is stored in the dataset. Set parameters always include `_set` at the end of the id. And lastly, you can't perform operations on dataset arrays without first changing them into a numpy array (using one of the two methods I showed above). This is because a dataset array contains extra information, and it also keeps the dataset from being changed accidentally.

If you need to rerun a Jupyter Notebook or want to recall a dataset in another file, use the function:

In [24]:
data1 = qc.load_data('data/2019-07-31/#001__02-12-18')
# where the string is the path to the file listed in "location" in the output from the parameter sweep.

`plot1` contains a list of the plots, in the same order as parameters are measured. You can display a plot in the Jupyter Notebook by entering `plot[index]` where index is an integer from 0 to n-1.

### One more measurement
to demonstrate a few more features. After (and only after) listing the comma-separated measured parameters, one can access a few keyword arguments (write them similar to keyword=value).

In [25]:
setarr = np.linspace(0, 5, 11)
data2, plot2 = npd.single_param_sweep(pmanual, setarr, 0.8, p1, instr1.c1, instr1.c2, instr1.p2, DataName='allparams', YParam=[p1, instr1.c2])

Started at 2019-07-31 02:13:11
DataSet:
   location = 'data/2019-07-31/#002_allparams_02-13-10'
   <Type>   | <array_id>  | <array.name> | <array.shape>
   Setpoint | pmanual_set | pmanual      | (11,)
   Measured | p1          | p1           | (11,)
   Measured | instr1_c1   | c1           | (11,)
   Measured | instr1_c2   | c2           | (11,)
   Measured | instr1_p2   | p2           | (11,)
Finished at 2019-07-31 02:13:27


Note that here only two parameters are plotted, as specified in the `YParam` keyword argument. plot2 only contains these two plots. However, the other parameters were still measured. The filename also includes a the string 'allparams'. The `DataName` keyword argument is good for including comments on things that aren't measured (the gain of an amplifier or something like that, for instance). You can also change the variable on the x-axis of the plot using `XParam`. You can also disallow all live plotting with the keyword argument `plot_results=False`, and you can disable saving the plots at the end by setting `save_plots=False`.

## 2. Demonstration of twod_param_sweep

There's not much different here. Just a couple subtleties to note.

Structure:

`data, plot = npd.twod_param_sweep(SetParam1, SetArray1, SetParam2, SetArray2, *MeasParams, SetDelay1=0, SetDelay2=0)`

The main difference is that there are two parameters to set. The y-axis plotted parameter is SetParam1, which changes after one cycle of SetParam2 (x-axis). The measured parameters are represented in the z-axis. Note here that the delay times are keyword arguments, and they must be placed after the comma-separated measured parameters.

Here, one can choose which parameters to plot using the keyword `ZParam=[params]`. `DataName` still works as well. You can also optionally choose to set SetParam2 to an intermediate value while SetParam2 is changing: use keyword argument `Param2_SetBetween=value`.

When the live plot is running, the plot will not update until a row is finished.

In [26]:
# First we need another settable parameter
pmanual2 = qc.Parameter('pmanual2', get_cmd=None, set_cmd=None)

In [32]:
pmanual2(0)

In [33]:
pmanual2.step = 0.5
pmanual2.inter_delay = 0.15

In [34]:
pmanual2(1)

In [35]:
setarr1 = np.linspace(0, 10, 6)
setarr2 = np.linspace(-2, 18, 21)
data3, plot3 = npd.twod_param_sweep(pmanual, setarr1, pmanual2, setarr2, p1, instr1.c1, instr1.p2, SetDelay1=1.5, SetDelay2=0.05)

Started at 2019-07-31 02:43:36
DataSet:
   location = 'data/2019-07-31/#004__02-43-32'
   <Type>   | <array_id>   | <array.name> | <array.shape>
   Setpoint | pmanual_set  | pmanual      | (6,)
   Setpoint | pmanual2_set | pmanual2     | (6, 21)
   Measured | p1           | p1           | (6, 21)
   Measured | instr1_c1    | c1           | (6, 21)
   Measured | instr1_p2    | p2           | (6, 21)
Finished at 2019-07-31 02:44:57


Demonstrate some of the other features in the keyword arguments

In [37]:
setarr1 = np.linspace(0, 10, 6)
setarr2 = np.linspace(-2, 18, 21)
data4, plot4 = npd.twod_param_sweep(pmanual, setarr1, pmanual2, setarr2, p1, instr1.c1, instr1.c2, instr1.p2, SetDelay1=1.5, SetDelay2=0.05, Param2_SetBetween=0, DataName='twod_run2', ZParam=[p1, instr1.c2], save_plots=False)

Started at 2019-07-31 02:49:02
DataSet:
   location = 'data/2019-07-31/#005_twod_run2_02-48-59'
   <Type>   | <array_id>   | <array.name> | <array.shape>
   Setpoint | pmanual_set  | pmanual      | (6,)
   Setpoint | pmanual2_set | pmanual2     | (6, 21)
   Measured | p1           | p1           | (6, 21)
   Measured | instr1_c1    | c1           | (6, 21)
   Measured | instr1_c2    | c2           | (6, 21)
   Measured | instr1_p2    | p2           | (6, 21)
Finished at 2019-07-31 02:50:33


## 3. Demonstration of data_log

The log function doesn't set any parameters. It just measures parameters after each delay time has passed.

Structure:

`data, plot = npd.data_log(delay(s), *MeasParams, N=None, minutes=time(min), DataName='', XParam=param,
                        YParam=[params], breakif=npd.breakat(breakparam, setpoint, epsilon))`
                        
After listing the measured parameters, one must specify either the number of points to measure (N) or the time (minutes) expected for the duration of the log.

Since there are no set parameters, if you do not specify a parameter in XParam, all the parameters will be plotted with respect to time since the start of the log. Time data is stored in a parameter called time0. You can also use a list for XParam, but it must be the same length as YParam.

Forget the breakif argument for the moment.

In [38]:
instr1.resetcounts() # set counts back to 0.

In [39]:
pmanual()

10.0

In [40]:
data5, plot5 = npd.data_log(1.2, p1, instr1.c1, instr1.c2, instr1.p2, minutes=0.9, XParam=instr1.c2)

Started at 2019-07-31 10:26:31
DataSet:
   location = 'data/2019-07-31/#006__10-26-28'
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | count_set  | count        | (45,)
   Measured | time0      | time0        | (45,)
   Measured | p1         | p1           | (45,)
   Measured | instr1_c1  | c1           | (45,)
   Measured | instr1_c2  | c2           | (45,)
   Measured | instr1_p2  | p2           | (45,)
Finished at 2019-07-31 10:27:30


You can see that 4 plots show up, each with instr1.c2 in the x axis. In the y axes are: instr1.c2, instr1.c1, p1, and instr1.p2. There are two other parameters listed above: count_set is a list of points from 0 to N. time0 is a list of times recorded right before the instruments are measured (measured in seconds after the time the log was started).

You can see how this could be useful if you want to measure the results of the slow progress of a parameter like temperature or magnetic field. You set the temperature/field right before starting the log.

In [57]:
print(data5.count_set)
s = '{:.4f}, '*(len(data5.time0) - 1) + '{:.4f}'
print('time0: [' + s.format(*data5.time0[:]) + ']')

DataArray[45]: count_set
array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
       40., 41., 42., 43., 44., 45.])
time0: [0.0038, 1.3341, 2.6589, 3.9626, 5.2837, 6.5984, 7.9344, 9.2451, 10.5507, 11.8503, 13.1826, 14.4914, 15.8207, 17.1577, 18.4826, 19.7935, 21.0954, 22.4118, 23.7485, 25.0542, 26.3657, 27.6804, 28.9851, 30.2881, 31.6009, 32.9063, 34.2358, 35.5314, 36.8351, 38.1383, 39.4501, 40.7448, 42.0434, 43.3197, 44.6097, 45.9143, 47.2150, 48.5025, 49.7962, 51.0848, 52.3789, 53.6674, 54.9615, 56.2553, 57.5482]


In [58]:
0.9*60

54.0

**Small detail:** the last time point is at 57.5 seconds instead of 54 seconds. The number of data points is calculated by N = minutes*60/delay. If measurement takes a finite amount of time, then it will take longer.

### Break conditions

Next, let's try utilizing the break condition. The breakif keyword takes a callable function with no arguments, returning true when (one that can be called with open parentheses like func() ). I made a function that breaks produces this based on:
- `breakparam`: The parameter that must reach a setpoint
- `setpoint`: The point it must reach (+/- epsilon) before breaking
- `epsilon`: The range of values around the setpoint such that the log will end if  |breakparam - setpoint| < epsilon

The other two optional keyword arguments you can use are
- `waitafter=`: Waits the specified time (s) after the break condition is reached
- `boolcond=`: This allows you to select a different boolean condition. (If not used or None, it works as described above)
    - `"lessthan"`: the break condition is returned when the breakparam is measured below the setpoint
    - `"greaterthan"`: the break condition is returned when the breakparam is measured as larger than the setpoint
    
For both of these, the epsilon argument is not used, though you still need to have a placeholder value in there. Note that you must have the `minutes` argument be at least as long or longer than the time taken to meet the break condition (the reason is given below).

In [59]:
instr1._count1

45

In [60]:
data6, plot6 = npd.data_log(1, p1, instr1.c1, instr1.c2, instr1.p2, minutes=5, breakif=npd.breakat(instr1.c1, 58, 2), XParam=[instr1.c1, instr1.c2], YParam=[p1, instr1.p2], save_plots=False)

Started at 2019-07-31 11:17:26
DataSet:
   location = 'data/2019-07-31/#007__11-17-25'
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | count_set  | count        | (300,)
   Measured | time0      | time0        | (300,)
   Measured | p1         | p1           | (300,)
   Measured | instr1_c1  | c1           | (300,)
   Measured | instr1_c2  | c2           | (300,)
   Measured | instr1_p2  | p2           | (300,)
Finished at 2019-07-31 11:17:33


You should see two plots with the x-axis parameter and y-axis parameter in the same order as they appear in the lists fed to XParam and YParam. Let's look at the break parameter array.

In [62]:
data6.instr1_c1

DataArray[300]: instr1_c1
array([46., 48., 50., 52., 54., 56., nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, n

The first thing to note is that it advances in steps of 2. This is because it was measured once to be recorded in the dataset and once for the breakparam. We could have allowed it to advance in steps of 1 by using instr1.c1.get_latest as the break parameter.

The second thing to notice is that the array is a length-300 array of mostly nan values. This is because a QCodes dataset (only in the legacy version) requires one to set up the array size at the start of the measurement. It starts out full of nan values (which won't be plotted) and replaces them as values are measured. This introduces one complication to the break condition (mentioned above): if minutes is too short, it will set up an array that is too small, and the array space will run out before the break condition is reached. Once again, set `minutes` to something longer than the expected time to reach the break condition.

In [64]:
# If you want to extract the values in an array that doesn't include nan values, you can use
firstnan = np.isnan(data6.instr1_p2[:]).argmax()
data6.instr1_p2[:firstnan]

array([102., 105., 108., 111., 114., 117.])

In [65]:
data6.instr1_p2[firstnan+1]

nan